# Imports and function definitions

In [ ]:
%matplotlib inline
import glob
import re
from itertools import cycle
from functools import partial
from os.path import basename, splitext, join

import matplotlib.pyplot as plt
import numpy as np
import typhon.plots
from matplotlib.legend_handler import HandlerTuple
from netCDF4 import Dataset

os.environ['PSRAD_PATH'] = '/scratch/uni/u237/users/lkluft/icon-aes/psrad/'
import conrad


plt.style.use(typhon.plots.styles('typhon'))


def get_legend_label(filename, label=None, **kwargs):
    """Get label for legend entry."""
    try:
        finfo = conrad.utils.extract_metadata(filename)
        experiment = finfo.experiment
    except:
        experiment = ''
    
    if 'layer' in experiment:
        label = f'{finfo.scale:>5} layers'
    elif 'co2' in experiment or 'shift' in experiment:
        label = rf'$\sf CO_2 \times {finfo.scale}$'
    elif label is None:
        label = basename(filename)
    else:
        label = basename(filename)
    
    return label
    

def scale_getter(filename):
    """Return the scale factor of a simulation."""
    return float(conrad.utils.extract_metadata(filename).scale)

# Surface temperature and radiation budget

In [ ]:
ATMOSPHERE = 'rce-rrtmg'
EXPERIMENT = 'scale-co2-coupledrh'
SCALE = '*'
EXTRA=''
RESULT_DIR='results/'
PLOT_DIR = 'plots/'
    
filepattern = conrad.utils.get_filepath(
    atmosphere=ATMOSPHERE,
    experiment=EXPERIMENT,
    scale=SCALE,
    extra=EXTRA,
    result_dir=RESULT_DIR,
    create_tree=False,
)

scenarios = sorted(glob.glob(filepattern), key=scale_getter, reverse=True)

if len(scenarios) == 0:
    raise FileNotFoundError('No input files found. Probably screwed glob pattern.')

fig, [ax, ax2] = plt.subplots(nrows=2, sharex=True)
for scenario in scenarios:
    with Dataset(scenario) as data:
        time = data.variables['time'][:] / 24  # time in days
        Ts = data.variables['temperature'][:]
        toa = data.variables['toa'][:]
    
    # Surface temperatures.
    ax.plot(
        time, Ts,
        label=get_legend_label(scenario),
        linewidth=3,
    )
    
    # Radiation budget at top of the atmosphere.
    ax2.plot(
        time, toa,
        linewidth=3,
    )

ax.grid('on')
ax2.grid('on')
ax.legend(loc='upper left', bbox_to_anchor=(1, 1))
ax2.set_xlabel('Day')
ax.set_xlim(left=1)
ax.set_ylabel('$T_s$ [K]')
ax2.set_ylabel('TOA [$Wm^{-2}$]')
fig.savefig(join(PLOT_DIR, f'{ATMOSPHERE}_{EXPERIMENT}_t_surface.pdf'))

# Plot equilibrium temperature profiles

## Pressure coordinates

In [ ]:
fig, ax = plt.subplots(figsize=typhon.plots.figsize(8, portrait=True))

# Equilibrium states for different CO2 scenarios
for scenario in scenarios:
    with Dataset(scenario) as data:
        p = data.variables['plev'][:]
        T = data.variables['T'][-1, :]
        contop = data.variables['convective_top_plev'][-1]
    
    print(f'Surface temperature {T[0]} K for {scale_getter(scenario)}.')
    l = typhon.plots.profile_p_log(
        p, T,
        label=get_legend_label(scenario),
        linewidth=4 if scale_getter(scenario) == 1 else 2,
        )
    
    # Mark top of convection (height in pressure).
    ax.axhline(contop, linestyle='dotted', color=l[0].get_color(), alpha=0.3)
    
# ERA5 climatology
clim = xml.load('/home/zmaw/u300509/lkluft/era5/results/t_climatology.xml')
p = clim.grids[1]
T = clim.get('temperature_mean', keep_dims=False)
print(f'ERA5 surface temperature {T[-1]} K.')
typhon.plots.profile_p_log(
    p, T,
    label='ERA5',
    color='black',
    linewidth=4,
)

ax.grid(True)
ax.legend(loc='upper right')
ax.set_xlabel('Temperature [K]')
ax.set_ylim(top=p.min(), bottom=p.max())
ax.set_xlim(left=180, right=310)

fig.savefig(join(PLOT_DIR, f'{ATMOSPHERE}_{EXPERIMENT}_t_profile.pdf'))

## Geopotential coordinates

In [ ]:
from scipy.interpolate import interp1d


fig, ax = plt.subplots(figsize=typhon.plots.figsize(8, portrait=True))
for scenario in scenarios:
    with Dataset(scenario) as data:
        z = np.array(data.variables['z'])[-1, :]
        p = np.array(data.variables['plev'])[:]
        T = data.variables['T'][-1, :]
        contop = data.variables['convective_top_plev'][-1]
        contop_z = interp1d(p, z)(contop)
    
    l = typhon.plots.profile_z(
        z, T,
        label=get_legend_label(scenario),
        linewidth=4 if scale_getter(scenario) == 1 else 2,
        )
    
    ax.axhline(contop_z, linestyle='dotted', color=l[0].get_color(), alpha=0.3)
    
ax.grid(True)
ax.legend(loc='upper right')
ax.set_xlabel('Temperature [K]')
ax.set_ylim(bottom=z.min(), top=z.max())
ax.set_xlim(left=180, right=310)

fig.savefig(join(PLOT_DIR, f'{ATMOSPHERE}_{EXPERIMENT}_t_profile_z.pdf'))

## Convective top height

In [ ]:
fig, ax = plt.subplots()

for n, scenario in enumerate(scenarios):
    with Dataset(scenario) as data:
        plev = np.array(data.variables['plev'])[:]
        time = np.array(data.variables['time'])[:] / 24
        ctop = data.variables['convective_top_plev'][:]
        
    ax.plot(time, ctop / 1e2, color=f"C{n}", label=get_legend_label(scenario))
    
ax.invert_yaxis()
ax.grid(True)
ax.legend(bbox_to_anchor=(1, 1))
ax.set_ylabel('Convective top [hPa]')
ax.set_xlabel('Days')
ax.set_xlim(left=0)

fig.savefig(join(PLOT_DIR, f'{ATMOSPHERE}_{EXPERIMENT}_convective_top_plev.pdf'))

fig, ax = plt.subplots()
for n, scenario in enumerate(scenarios):
    with Dataset(scenario) as data:
        plev = np.array(data.variables['plev'])[:]
        time = np.array(data.variables['time'])[:] / 24
        ctop_T = data.variables['convective_top_temperature'][:]
        
    ax.plot(time, ctop_T, color=f"C{n}", label=get_legend_label(scenario))
    
ax.grid(True)
ax.legend(bbox_to_anchor=(1, 1))
ax.set_ylabel('Convective top temperature [K]')
ax.set_xlabel('Days')
ax.set_xlim(left=0)

fig.savefig(join(PLOT_DIR, f'{ATMOSPHERE}_{EXPERIMENT}_convective_top_temperature.pdf'))

# PHAT mechanism

In [ ]:
def plot_diagonal(ax=None, **kwargs):
    """Plot the bisectrix while preserving old axis limits."""
    if ax is None:
        ax = plt.gca()
    
    xmin, xmax = ax.get_xlim()
    l = ax.plot([xmin, xmax], [xmin, xmax], **kwargs)
    ax.set_xbound(xmin, xmax)
    
    return l


controlparams = [
    # ('scale-co2-nopeak', 'o', 'lightgrey', 'No peak'),
    # ('scale-co2-fixedrh', '^', 'skyblue', 'Decoupled troposphere'),
    # ('scale-co2-coupledrh', 'D', 'sandybrown', 'Fully coupled'),
    # ('scale-co2-fixedstrato', 'v', 'mediumseagreen', 'Decoupled stratosphere'),
    ('scale-co2-nopeak', 'o', 'black', 'No peak'),
    ('scale-co2-fixedrh', '^', 'grey', 'Decoupled troposphere'),
    ('scale-co2-coupledrh', 'D', 'silver', 'Fully coupled'),
    ('scale-co2-fixedstrato', 'v', 'whitesmoke', 'Decoupled stratosphere'),
]


fig, ax = plt.subplots(figsize=typhon.plots.figsize(8))
# colors = cycle(f'C{i}' for i in range(4))
handle_groups = []
for experiment, marker, color, _ in controlparams:
    filepattern = conrad.utils.get_filepath(
        atmosphere='rce-rrtmg-nopeak' if 'nopeak' in experiment else 'rce-rrtmg',
        experiment=experiment,
        result_dir=RESULT_DIR,
        create_tree=False,
    )
    
    handles = []
    # color = next(colors)
    for scenario in sorted(glob.glob(filepattern), key=scale_getter, reverse=True):
        with Dataset(scenario) as data:
            t_surface = data.variables['temperature'][-1]
            t_convective_top = data.variables['convective_top_temperature'][-1]

            t_s_ref = data.variables['temperature'][0]
            t_con_ref = data.variables['convective_top_temperature'][0]
    
        handle, = ax.plot(
            t_surface - t_s_ref,
            t_convective_top - t_con_ref,
            marker=marker,
            markeredgecolor='k',
            markersize=12,
            linestyle='none',
            color=color,
            label=get_legend_label(scenario),
        )
        handles.append(handle)
    handle_groups.append(handles)
    
ax.grid(True)
ax.axhline(0, color='#b0b0b0', linewidth=1.5, zorder=-10)
ax.axvline(0, color='#b0b0b0', linewidth=1.5, zorder=-10)
plot_diagonal(ls='--', color='#b0b0b0', linewidth=0.8, zorder=-10)

ax.legend(
    [tuple(g)[0] for g in handle_groups],
    [p[3] for p in controlparams],
    handler_map={tuple: HandlerTuple(ndivide=None)},
    fontsize='x-small',
)
ax.set_aspect('equal')
ax.set_xticks(np.arange(-2, 9, 2))
ax.set_xlabel(r'$\Delta\sf T_{surface} [K]$')
ax.set_ylabel('$\Delta\sf T_{convection}$ [K]')

fig.savefig(join(PLOT_DIR, f'phat_mechanism2.pdf'))

# Gregory plot

In [ ]:
fig, ax = plt.subplots(figsize=typhon.plots.figsize(8))
for scenario in scenarios:
    with Dataset(scenario) as dataset:
        T_sfc = dataset.variables['temperature'][:]
        toa = dataset.variables['toa'][:]
    
    ax.plot(
        T_sfc - T_sfc[0], toa,
        label=get_legend_label(scenario),
        marker='.',
        linestyle='none',
        rasterized=True,
    )
    
    slope, offset = np.polyfit(T_sfc - T_sfc[0], toa, 1)
    ecs = T_sfc[-1] - T_sfc[0]
    print(get_legend_label(scenario), f'ECS: {ecs:.2f} K', f'Feedback: {-1/slope:.3f} K/(W/m^2)')

ax.grid(True)
ax.axhline(0, color='#b0b0b0', linewidth=1.5, zorder=-10)
ax.axvline(0, color='#b0b0b0', linewidth=1.5, zorder=-10)

ax.legend(loc='upper left', fontsize='x-small')
ax.set_xlabel('Surface temperature change [K]')
ax.set_xlim(-3, 8)
ax.set_xticks(np.arange(np.floor(ax.get_xlim()[0]), np.ceil(ax.get_xlim()[1])))
ax.set_ylabel(r'TOA imbalance [$\sf W/m^2$]')
ax.set_ylim(-6, 16)
ax.set_yticks(np.arange(np.floor(ax.get_ylim()[0]), np.ceil(ax.get_ylim()[1]), 3))

fig.savefig(join(PLOT_DIR, f'{ATMOSPHERE}_{EXPERIMENT}_gregory.pdf'), dpi=300)

# Plot changes in water vapor mixing ratios

In [ ]:
scenario = scenarios[0]

fig, ax = plt.subplots()
fig.suptitle(get_legend_label(scenario))
with Dataset(scenario) as data:
    time = data.variables['time'][:] / 24
    q = data.variables['H2O'][:]
    p = data.variables['plev'][:]
    T = data.variables['T'][:]
    
# Calculate relative humidity.
RH = 100 * typhon.atmosphere.relative_humidity(q, p, T)

sm = ax.pcolormesh(
    time, p, RH.T,
    cmap=plt.get_cmap('density', 10),
    rasterized=True,
    vmin=0,
    vmax=100,
)
cb = fig.colorbar(sm, label='Relative humidity [%]')
cb.set_ticks(np.arange(0, 101, 10))
ax.invert_yaxis()
ax.set_yscale('log')
ax.set_ylabel('Pressure [hPa]')
ax.set_ylim(p.max(), 50e2)
ax.set_xlabel('Days')
ax.yaxis.set_major_formatter(typhon.plots.HectoPascalLogFormatter())
ax.yaxis.set_minor_formatter(typhon.plots.HectoPascalLogFormatter())

fig.savefig(join(PLOT_DIR, f'{ATMOSPHERE}_{EXPERIMENT}_rh_profile.pdf'), dpi=300)

# Layer thickness

In [ ]:
fig2, ax2 = plt.subplots()
for scenario in scenarios:
    with Dataset(scenario) as dataset:
        plev = dataset.variables['plev'][:]
        time = dataset.variables['time'][:] / 24
        z_field = dataset.variables['z'][:]
        
    thickness = np.diff(z_field, axis=1)

    fig, ax = plt.subplots(figsize=(10, 6))
    change_in_thickness = thickness - thickness[0, :]
    rel_change_in_thickness = (thickness - thickness[0, :]) / thickness[0, :] * 100
    sm = ax.pcolormesh(time, plev[:-1], rel_change_in_thickness.T[:, :-1],
                       cmap='difference', rasterized=True, vmin=-15, vmax=5)
    ax.invert_yaxis()
    ax.set_yscale('log')
    ax.set_ylabel('Pressure [hPa]')
    ax.set_ylim(plev.max(), plev[:-1].min())
    ax.set_xlabel('Days')
    ax.set_xlim(left=0)
    ax.yaxis.set_major_formatter(typhon.plots.HectoPascalLogFormatter())
    ax.yaxis.set_minor_formatter(typhon.plots.HectoPascalLogFormatter())
    cb = fig.colorbar(sm, label='Change in layer thickness [%]')
    cb.set_clim(-15, 15)
    typhon.plots.center_colorbar(cb)
    ax.set_title(get_legend_label(scenario))

    ax2.plot(time, np.sum(change_in_thickness.T, axis=0) / 1e3,
             linewidth=3,
             label=get_legend_label(scenario))
    
ax2.set_title(get_legend_label(scenario))
ax2.set_ylabel('Change in atmosphere thickness [km]')
ax2.set_xlabel('Days')
ax2.set_xlim(left=0)
ax2.grid(True)
ax2.legend(loc='upper left', bbox_to_anchor=(1, 1))

# fig2.savefig(join(PLOT_DIR, f'{ATMOSPHERE}_{EXPERIMENT}_atmosphere_thickness.pdf'))